# Vector Database Operations

A simple notebook for adding and viewing vector data in PostgreSQL with pgvector.

## 1. Setup and Imports

In [1]:
# Import required libraries
import psycopg2
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import json

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Database Connection

In [2]:
# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'rag_db',
    'user': 'rag_user',
    'password': 'rag_password'
}

def connect_db():
    """Connect to PostgreSQL database"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Connected to database successfully!")
        return conn
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None

# Create connection
conn = connect_db()

# Create SQLAlchemy engine for pandas
db_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(db_url)

✅ Connected to database successfully!


## 3. Add Vector Data

In [4]:
def add_sample_data(content, vector_data, metadata=None):
    """Add sample data with vector embedding"""
    if not conn:
        print("❌ No database connection")
        return False
    
    try:
        with conn.cursor() as cursor:
            # Convert vector to PostgreSQL format
            vector_str = '[' + ','.join(map(str, vector_data)) + ']'
            
            # Insert into sample_data table
            cursor.execute("""
                INSERT INTO sample_data (data, embedding) 
                VALUES (%s, %s::VECTOR)
                RETURNING id
            """, (json.dumps({
                'content': content,
                'type': 'sample',
                'metadata': metadata or {}
            }), vector_str))
            
            new_id = cursor.fetchone()[0]
            conn.commit()
            
            print(f"✅ Added sample data with ID: {new_id}")
            return new_id
            
    except Exception as e:
        print(f"❌ Error adding data: {e}")
        conn.rollback()
        return False

def add_document_embedding(content, vector_data, metadata=None):
    """Add document with vector embedding"""
    if not conn:
        print("❌ No database connection")
        return False
    
    try:
        with conn.cursor() as cursor:
            # Convert vector to PostgreSQL format
            vector_str = '[' + ','.join(map(str, vector_data)) + ']'
            
            # Insert into document_embeddings table
            cursor.execute("""
                INSERT INTO document_embeddings (content, embedding, metadata) 
                VALUES (%s, %s::VECTOR, %s)
                RETURNING id
            """, (content, vector_str, json.dumps(metadata or {})))
            
            new_id = cursor.fetchone()[0]
            conn.commit()
            
            print(f"✅ Added document embedding with ID: {new_id}")
            return new_id
            
    except Exception as e:
        print(f"❌ Error adding document: {e}")
        conn.rollback()
        return False

# Example usage
print("📝 Adding sample data...")

# Add sample data (3D vector)
sample_id = add_sample_data(
    content="This is a sample document for testing",
    vector_data=[0.1, 0.2, 0.3],
    metadata={'source': 'notebook', 'category': 'test'}
)

# Add document embedding (5D vector)
doc_id = add_document_embedding(
    content="Another document with different content for similarity testing",
    vector_data=[0.4, 0.5, 0.6, 0.7, 0.8],
    metadata={'source': 'notebook', 'category': 'example'}
)

📝 Adding sample data...
✅ Added sample data with ID: f2aa41e4-d084-4c0f-9169-4f4bbb91713c
✅ Added document embedding with ID: a7d74c04-214e-4b6b-b960-d87874e1a3c1


## 4. View Data

In [6]:
def view_sample_data(limit=10):
    """View sample data from database"""
    if not conn:
        print("❌ No database connection")
        return
    
    try:
        df = pd.read_sql("""
            SELECT id, data, created_at, 
                   vector_extension_available,
                   embedding::text as embedding
            FROM sample_data 
            ORDER BY created_at DESC 
            LIMIT %s
        """, engine, params=[limit])
        
        print(f"📊 Sample Data ({len(df)} records):")
        display(df)
        return df
        
    except Exception as e:
        print(f"❌ Error viewing data: {e}")
        return None

def view_document_embeddings(limit=10):
    """View document embeddings from database"""
    if not conn:
        print("❌ No database connection")
        return
    
    try:
        df = pd.read_sql("""
            SELECT id, content, 
                   embedding::text as embedding,
                   metadata, created_at
            FROM document_embeddings 
            ORDER BY created_at DESC 
            LIMIT %s
        """, engine, params=[limit])
        
        print(f"📊 Document Embeddings ({len(df)} records):")
        display(df)
        return df
        
    except Exception as e:
        print(f"❌ Error viewing embeddings: {e}")
        return None

def view_all_data():
    """View all data from both tables"""
    print("=== SAMPLE DATA ===")
    sample_df = view_sample_data()
    
    print("\n=== DOCUMENT EMBEDDINGS ===")
    embeddings_df = view_document_embeddings()
    
    return sample_df, embeddings_df

# View the data
sample_data, doc_embeddings = view_all_data()

=== SAMPLE DATA ===
❌ Error viewing data: List argument must consist only of tuples or dictionaries

=== DOCUMENT EMBEDDINGS ===
❌ Error viewing embeddings: List argument must consist only of tuples or dictionaries


## 5. Basic Similarity Search

In [ ]:
def find_similar_documents(query_vector, limit=5):
    """Find documents similar to query vector"""
    if not conn:
        print("❌ No database connection")
        return
    
    try:
        # Convert query vector to PostgreSQL format
        query_str = '[' + ','.join(map(str, query_vector)) + ']'
        
        with conn.cursor() as cursor:
            # Find similar documents using cosine similarity
            cursor.execute("""
                SELECT id, content, 
                       1 - (embedding <=> %s::VECTOR) as similarity,
                       embedding::text as embedding
                FROM document_embeddings 
                ORDER BY embedding <=> %s::VECTOR
                LIMIT %s
            """, (query_str, query_str, limit))
            
            results = cursor.fetchall()
            
            print(f"🔍 Similar documents to query {query_vector}:")
            print("-" * 60)
            
            for row in results:
                doc_id, content, similarity, embedding = row
                print(".4f")
                print(f"   Content: {content[:50]}...")
                print()
                
            return results
            
    except Exception as e:
        print(f"❌ Error in similarity search: {e}")
        return None

# Example similarity search
print("🔍 Testing similarity search...")

# Search using the vector we added earlier
query_vector = [0.4, 0.5, 0.6, 0.7, 0.8]
similar_docs = find_similar_documents(query_vector, limit=3)

# Search with a different vector
print("\n🔍 Testing with different query vector...")
different_query = [0.2, 0.3, 0.4, 0.5, 0.6]
similar_docs2 = find_similar_documents(different_query, limit=3)

## 6. Cleanup

In [ ]:
# Close connections
if conn:
    conn.close()
    print("🔌 Database connection closed.")

if 'engine' in locals():
    engine.dispose()
    print("🔌 SQLAlchemy engine disposed.")

print("✅ Notebook completed successfully!")